In [5]:
%matplotlib widget
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.image as IM
from matplotlib import rc
from matplotlib.lines import Line2D
from skimage import color
import cv2 as cv
import glob
import time
import scipy.misc, scipy.stats
import sys
import csv
import os
import numpy.ma as ma
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Image
import matplotlib.colors as colors
import matplotlib.patches as patches
from PIL import Image
from PIL import ImageOps
from scipy.ndimage import gaussian_filter
from tqdm.notebook import trange, tqdm, tnrange

import matplotlib as mpl
mpl.rc('font',family='Times New Roman')

from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [125]:
def FindBestk(k_distance, magnification, pixel):
    possible_ks = [3,5,7,9,11,13,15,17,19,21]
    ideal_k = int(np.round(k_distance/(pixel/magnification),0))
#     print('ideal k: ', ideal_k)
    k_diff = 21
    for kk in possible_ks:
        diff = np.absolute(kk - ideal_k)
        if diff<k_diff:
            k_best = kk
            k_diff = diff
    return k_best, k_diff

def DecToStr(decimal):
    decimal_str = str(decimal)
    decimal_int, decimal_dec = decimal_str.split('.')
    return decimal_int+'p'+decimal_dec

def Sobel(im, k, N):
    (nrows, ncols) = im.shape
    sobelx = cv.Sobel(im,ddepth=cv.CV_64F, dx=1,dy=0,ksize=k) #cv.CV_64F
    sobely = cv.Sobel(im,ddepth=cv.CV_64F, dx=0,dy=1,ksize=k)
    sobel = np.zeros(im.shape)
    if N=='self':
        sobel = np.sqrt(sobelx**2 + sobely**2)
        sobel = sobel/np.amax(sobel)
    else: 
        sobel = np.sqrt(sobelx**2 + sobely**2)/N
    return sobel
    
def SeparateInCells(im, cell_x, cell_y, max_used):
    max_used = int(max_used)
    (nrows, ncols) = im.shape
    Nx_cell, Ny_cell = int(ncols/cell_x), int(nrows/cell_y)
    leftout_x = ncols - int(Nx_cell*cell_x)
    leftout_y = nrows - int(Ny_cell*cell_y)
    if leftout_x>=0.5*cell_x:
        Nx_cell+=1
        leftout_x = ncols - int(Nx_cell*cell_x)
        last_x_cell_imcomplete = True
    else:
        last_x_cell_imcomplete = False
    if leftout_y>=0.75*cell_y:
        Ny_cell+=1
        leftout_y = nrows - int(Ny_cell*cell_y)
        last_y_cell_imcomplete = True
    else:
        last_y_cell_imcomplete = False
#     print('leftovers = ('+str(leftout_x)+', '+str(leftout_y)+')')
    sharpness_map = np.zeros(im.shape)
    for i in range(0,Nx_cell):
        for j in range (0,Ny_cell):
            if (j==(Ny_cell-1) and i==(Nx_cell-1)):
                cell = im[(cell_y*j):,(cell_x*i):]
            elif j==(Ny_cell-1):
                cell = im[(cell_y*j):,(cell_x*i):(cell_x*(i+1))]
            elif i==(Nx_cell-1):
                cell = im[(cell_y*j):(cell_y*(j+1)),(cell_x*i):]
            else:
                cell = im[(cell_y*j):(cell_y*(j+1)),(cell_x*i):(cell_x*(i+1))]
            temp_list = cell.flatten()
            temp_list_sorted = sorted(temp_list, reverse=True)
            if len(temp_list_sorted)>= 2*max_used:
                max_cell = temp_list_sorted[max_used]
            else:
                max_cell = temp_list_sorted[int(len(temp_list_sorted)/2)]
            for ii in range(0,len(cell[0,:])):
                for jj in range(0,len(cell[:,0])):
                    working_x = cell_x*i + ii
                    working_y = cell_y*j + jj
                    sharpness_map[working_y,working_x]=max_cell
    return sharpness_map

def GetMask(Sharpness_Map, t):
    array1 = np.zeros(Sharpness_Map.shape)+1.0
    Mask = np.zeros(Sharpness_Map.shape)+1.0
    if np.amax(Sharpness_Map)>t:
        (y_mask, x_mask) = np.where(Sharpness_Map>t)
        for i in range(0,len(y_mask)):
            yy = y_mask[i]
            xx = x_mask[i]
            Mask[yy,xx] = 0.0
        array_masked = np.ma.array(array1, mask=Mask)
    else:
        array_masked = np.ma.array(array1, mask=Mask)
    return array_masked


def AddArtificialFeature(im, width_0_curved, start_0_curved):
    im_withFeature = im
    width_256_curved = width_0_curved
    p0 = start_0_curved
    p1 = start_0_curved+width_0_curved
    p2 = start_0_curved+width_0_curved+width_256_curved
    p3 = start_0_curved+width_0_curved*2+width_256_curved
    im_withFeature[p0:p3,p0:p3] = 0.0 #np.amin(im)
    im_withFeature[p1:p2,p1:p2] = 2.0**12 #np.amax(im)
    return im_withFeature

def RemoveArtificialFeature(sobel, width_0_curved, start_0_curved, k):
    sobel_removed = sobel
    width_256_curved = width_0_curved
    p0 = start_0_curved - k
    p3 = start_0_curved+width_0_curved*2+width_256_curved + k
#     sobel_removed[p0:p3,p0:p3] = np.amin(sobel)
    sobel_removed[p0:p3,p0:p3] = np.average(sobel[p3:(2*p3),p0:p3])
    return sobel_removed


def SeparateInCells(im, cell_x, cell_y):
    (nrows, ncols) = im.shape
    Nx_cell, Ny_cell = int(ncols/cell_x), int(nrows/cell_y)
    leftout_x = ncols - int(Nx_cell*cell_x)
    leftout_y = nrows - int(Ny_cell*cell_y)
    if leftout_x>=0.5*cell_x:
        Nx_cell+=1
        leftout_x = ncols - int(Nx_cell*cell_x)
        last_x_cell_imcomplete = True
    else:
        last_x_cell_imcomplete = False
    if leftout_y>=0.75*cell_y:
        Ny_cell+=1
        leftout_y = nrows - int(Ny_cell*cell_y)
        last_y_cell_imcomplete = True
    else:
        last_y_cell_imcomplete = False
#     print('leftovers = ('+str(leftout_x)+', '+str(leftout_y)+')')
    return Nx_cell, Ny_cell

def GetCell(im, nx, ny, cell_x, cell_y, Nx_cell, Ny_cell):
    if (ny==(Ny_cell-1) and nx==(Nx_cell-1)):
        cell = im[(cell_y*ny):,(cell_x*nx):]
    elif ny==(Ny_cell-1):
         cell = im[(cell_y*ny):,(cell_x*nx):(cell_x*(nx+1))]
    elif nx==(Nx_cell-1):
        cell = im[(cell_y*ny):(cell_y*(ny+1)),(cell_x*nx):]
    else:
        cell = im[(cell_y*ny):(cell_y*(ny+1)),(cell_x*nx):(cell_x*(nx+1))]
    return cell

def GetNMax(cell, Nmax):
    temp_list = cell.flatten()
    temp_list_sorted = sorted(temp_list, reverse=True)
    if Nmax>len(temp_list):
        print('N max larger than cell size. Using real max.')
        M = temp_list_sorted[0]
    elif Nmax>int(len(temp_list)/2):
        print('N max larger than half cell size. Consider using smaller N max.')
        M = temp_list_sorted[Nmax]
    else:
        M = temp_list_sorted[Nmax]
    return M

def GetNMax_perc(cell, NmaxPerc):
    temp_list = cell.flatten()
    Nmax = int(NmaxPerc*len(temp_list))
    temp_list_sorted = sorted(temp_list, reverse=True)
    if Nmax>len(temp_list):
        print('N max larger than cell size. Using real max.')
        M = temp_list_sorted[0]
    elif Nmax>int(len(temp_list)/2):
        print('N max larger than half cell size. Consider using smaller N max.')
        M = temp_list_sorted[Nmax]
    else:
        M = temp_list_sorted[Nmax]
    return M

def LoadImPerfect(path, M):
    im_perfect = Image.open(path)
    im_perfect = ImageOps.grayscale(im_perfect)
    im_perfect = np.array(im_perfect).astype('float64')
    ## Rescale im_perfect to 12 bits
    im_perfect = im_perfect/np.amax(im_perfect)*M-1 # counting the 0
    return im_perfect

def Range(cell):
    M = np.amax(cell)
    m = np.amin(cell)
    val = M-m
    return val

def Range_Perc(cell, NPerc):
    cell_list = cell.flatten()
    cell_list = sorted(cell_list, reverse=False)
    MMval = int(len(cell_list)*(1.0-NPerc))
    mmval = int(len(cell_list)*(NPerc))
    MM = cell_list[MMval]
    mm = cell_list[mmval]
    val = MM-mm
#     print(MM, mm, val)
    return val

def GetSigma(sobel_M, Michelson, a, b, c, m, n, d):
    return (-1*np.log((sobel_M-d)/(a*(m*Michelson+n))) + c)/b


def AdjustImPerfect(im, new_min, new_max):
    # im b/w [old_min, old_max], want between [new_min, new_max]
    new_im = im-np.amin(im) ## [0, old_max - old_min]
    new_im = new_im/np.amax(new_im) ## [0,1]
    new_im = new_im*(new_max - new_min) ##  [0, new_max - new_min]
    new_im = new_im + new_min ## [new_min, new_max]
    return new_im


def FindBestSigma(sobel_val, im_perfect_sub, new_min, new_max, width, start, kk):
    ## Rescale perfect image
    im_perfect_sub_rescaled = AdjustImPerfect(im_perfect_sub, new_min, new_max)
    # Define blurring params (sigma for Gaussian blurring)
    blur_sigma_list = np.arange(0.0, 25.0, 0.05)
    sobel_diff = 1.0 ## Theoretical largest difference
    for sigma in blur_sigma_list:
        im_blurred = gaussian_filter(im_perfect_sub_rescaled, sigma)
        ## add artificial feature that normalizes the sobel
        im_blurred_AF = AddArtificialFeature(im_blurred, width, start)
        ## take sobel
        im_blurred_sobel = Sobel(im_blurred_AF, kk, 'self')
        ## remove artificial featre
        im_blurred_sobel_noAF = RemoveArtificialFeature(im_blurred_sobel, width, start, kk)
        ## take value of max sobel
        MM_sobel = np.round(np.amax(im_blurred_sobel_noAF),8)
        new_sobel_diff = np.absolute(MM_sobel-sobel_val)
        if new_sobel_diff<sobel_diff:
            sobel_diff = new_sobel_diff
            best_sigma = sigma
    return best_sigma, sobel_diff  

def SigmaFromFit(sobel, gg):
    x,y = sobel, gg
    x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11 = 3.86586453e-01,  7.45356568e-02, -1.58228085e-01,\
    7.26188986e-02, -4.61201151e-01,  1.54735015e+01, -2.95744213e+01, -1.93291031e+02, \
    4.63108684e+02,  1.01242119e+03, -3.70339238e+03,  2.78189434e+03
    y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11 = -1.37834750e-01,  6.20813432e+00,  2.83267585e+00,\
    1.36391577e+02, -7.14824149e+01, -1.13349916e+02,  1.66504399e+02, -7.79616727e+01,\
    1.22507725e+01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00
    b = -1.61141766e+00
    A = x1 + x2*(x-x0) + x3*(x-x0)**2 + x4*(x-x0)**3 + x5*(x-x0)**4 + x6*(x-x0)**5 +\
    x7*(x-x0)**6 + x8*(x-x0)**7 + x9*(x-x0)**8 + x10*(x-x0)**9 + x11*(x-x0)**10
    B = y1 + y2*(y-y0) + y3*(y-y0)**2 + y4*(y-y0)**3 + y5*(y-y0)**4 + y6*(y-y0)**5 +\
    y7*(y-y0)**6 + y8*(y-y0)**7 #
    + y9*(y-y0)**8 + y10*(y-y0)**9 + y10*(y-y0)**9
    return A*B + b

def Max_Perc(cell, NPerc):
    cell_list = cell.flatten()
    cell_list = sorted(cell_list, reverse=True)
    MMval = int(len(cell_list)*(NPerc))
    MM = cell_list[MMval]
    return MM

def Min_Perc(cell, NPerc):
    cell_list = cell.flatten()
    cell_list = sorted(cell_list, reverse=False)
    mmval = int(len(cell_list)*(NPerc))
    mm = cell_list[mmval]
    return mm  

def GetDarkFromPath(dark_path, isFlat, isCompressed):
    dark_files = glob.glob(dark_path)
    dark = []
    if isFlat:
        for d in dark_files:
            dark_i = Image.open(d)
            dark.append(np.array(dark_i).astype('float64'))
    else:
        for d in dark_files:
            if isCompressed:
                dark_i = np.load(d)['arr_0']
            else:
                 dark_i = np.load(d)
            dark.append(dark_i)
    dark = np.array(dark)
    dark_avg = np.average(dark,axis=0)
#     dark_std = np.std(dark)
    return dark_avg

def InvertRows(im):
    (YY, XX) = im.shape
    imNew = np.zeros(im.shape)
    for j in range(0, int(YY/2)):
        imNew[2*j,:] = im[2*j+1,:]
        imNew[2*j+1,:] = im[2*j,:]
    return imNew

def MaskIm(im, D): ## D in pix
    (YY,XX) = im.shape
    x0, y0 = int(XX/2), int(YY/2)
    mask = np.zeros(im.shape) 
    for i in range(0,XX):
        for j in range(0,YY):
            r = np.sqrt( (i-x0)**2 + (j-y0)**2 )
            if r>(D/2):
                mask[j,i] = 1
    return ma.masked_array(im, mask=mask)

def RescaleImage(im, m, M): ## im between 0 and 1
    im_N = np.subtract(im, np.nanmin(im)) #im - np.amin(im)
    im_N = np.divide(im, np.nanmax(im_N)) #im_N/np.amax(im_N)
    im_N[np.where(im_N<m)] = m
    im_N[np.where(im_N>M)] = M
    im_N = np.subtract(im_N, np.nanmin(im_N)) #im_N-m
    im_N = np.divide(im_N,np.nanmax(im_N)) #im_N/M
    im_N[np.where(np.isnan(im))] = np.nan
    return im_N * 4095 ## 12 bit depth

def ComputeDark(darkPath):
    darks_paths = glob.glob(darkPath)
    Ndarks = len(darks_paths)
    print(f'There are {Ndarks} darks')
    Darks = []
    for n in range(0,Ndarks):
        dark = np.load(darks_paths[n])['arr_0'].astype('float64')
        Darks.append(dark)
    Darks = np.array(Darks)
    dark_avg = np.average(Darks,axis=0)
    dark_std = np.std(Darks, axis=0)
    return dark_avg, dark_std


In [1]:
"""

Define the paths

"""


focus_im_path = 'PATH_TO_IMAGE'
saving_path = 'PATH_TO_SAVING_DIRECTORY'

saturated_path = 'PATH_TO_WHITE_IMAGE'
dark_path = 'PATH_TO_DARK_IMAGES_*.npz'


path_im_perfect_curved = 'DotsSim_Curved.bmp'
path_im_perfect_flat = 'DotsSim_Flat.bmp'

In [ ]:
"""

Set options for the analysis


"""


isFlat = False          ## If analyzing images from the flat setup, handle the different format
isCompressed = True     ## If the numpy arrays are compressed (True for all recent data)
maskChamber = True      ## Mask the area outside the circular chamber (Curved sensor only)
MaskDust = True         ## If to use the white image to mask pieces of dust in the image
DivideDark = True       ## Normalize by darks
Inverted = False        ## For data gathered with the original code from apertus, fix line flip erro (already accounted for in GUI)

MaskReflections = False ## Mask areas (4) in the image (bad reflections ruining the analysis)
##White sphere mount2
x1a, x1b, y1a, y1b = 1700, 2400, 1900, 2550
x2a, x2b, y2a, y2b = 2400,3050, 1200, 1900
x3a, x3b, y3a, y3b = 1650, 2500, 500, 1200
x4a, x4b, y4a, y4b = 950, 1700, 1100, 1900

white_path = saturated_path

In [632]:
"""

Set parameters (in physical units)


"""

D=18 ##mm                      ## Chamber diameter, in mm

k_distance = 55 ##um           ## Kernel size for the sobel operator
cell_distance = 2.0#/2 ##mm    ## Cell size
AF_distance = 160 ##um         ## Artificial feature size (for sobel normalizaation)

curved_magnification = 0.89    ## Magnification for the curved sensor instrument
flat_magnification = 1.0       ## Magnificatino for the flat sensor instrument  

curved_pixel = 5.5 #um         ## Size of a pixel in the curved sensor instrument
flat_pixel = 8.0 ##um          ## Size of a pixel in the flat sensor instrument

shiftN_distance = 0.62 ##mm    ## Rolling cells method: shift size

sobel_mask_t = 0.08            ## Dust masking: Sobel threshold 

curved_width = 4096            ## Width (pixels) of the curved sensor
curved_height = 3072           ## Heigth (pixels) of the curved sensor
flat_width = 1312              ## Width (pixels) of the flat sensor
flat_height = 1082             ## Heigth (pixels) of the flat sensor

curved_white_level = 3243      ## Saturation level for the curved sensor 
                               ## (not exactly 12 bits - depends on sensor settings through cmv_reg)


## Sharpness maps parameters, depends on the setup
## sobel_max: set to 0 if taking the absolute maximum sobel value in the cell, >0 if taking a lower value 
##            (0.01 = 1% of the max) - allows to filter out dust that might have been missed by the mask
## greyscale_min, greyscale_max: in percentage of sorted pixels, min and max greyscale values in the cell
##                               also to avoid effects of noise/dust/dead pixels
## greyscale_min_largeCell: greyscale_min for edge cells that are larger than normal cells
if isFlat:
    sobel_max = 0.005
    greyscale_min, greyscale_max = 0.0000001, 0.001 #0.00014
    greyscale_min_largeCell = 0.00007
else:
    sobel_max = 0.02
    greyscale_min, greyscale_max =  0.001, 0.0005 #0.00014
    greyscale_min_largeCell = 0.001
    


## *** *** *** *** *** *** *** *** ***

print(f'SET PARAMETERS: \n')

## Magnification
print(f'magnification --> flat: {flat_magnification}, curved: {curved_magnification}\n')

## Cell size
flat_cell = int(np.round((cell_distance*10**3)/(flat_pixel/flat_magnification),0))
flat_cell_x, flat_cell_y = flat_cell, flat_cell
curved_cell = int(np.round((cell_distance*10**3)/(curved_pixel/curved_magnification),0))
curved_cell_x, curved_cell_y = curved_cell, curved_cell
print(f'cell size --> flat: {flat_cell}, curved: {curved_cell}\n')

## k
flat_sobel_k, flat_sobel_k_diff = FindBestk(k_distance, flat_magnification, flat_pixel)
curved_sobel_k, curved_sobel_k_diff = FindBestk(k_distance, curved_magnification, curved_pixel)
print(f'flat --> k: {flat_sobel_k}, difference from ideal: {flat_sobel_k_diff}')
print(f'curved --> k: {curved_sobel_k}, difference from ideal: {curved_sobel_k_diff}\n')

## Artificial feature width
width_0_flat = int(np.round(AF_distance/(flat_pixel/flat_magnification),0))
width_0_curved = int(np.round(AF_distance/(curved_pixel/curved_magnification),0))
print(f'artificial feature width --> flat: {width_0_flat}, curved: {width_0_curved}\n')

## Chamber masking
D_pix = D/(curved_pixel/curved_magnification*10**-3)
print(f'Chamber diameter --> {D}mm = {D_pix:.0f} pixels \n')

## Rolling sobel
shiftN_curved = int(np.round((shiftN_distance*10**3)/(curved_pixel/curved_magnification),0))
shiftN_flat = int(np.round((shiftN_distance*10**3)/(flat_pixel/flat_magnification),0))
print(f'Rolling sobel shift --> flat: {shiftN_flat}, curved: {shiftN_curved}\n')
        
if isFlat:
    curved_cell, curved_cell_x, curved_cell_y = flat_cell, flat_cell_x, flat_cell_y
    curved_sobel_k, curved_sobel_k_diff = flat_sobel_k, flat_sobel_k_diff
    width_0_curved = width_0_flat
    shiftN = shiftN_flat
#     sobel_mask_t = 0.006
    print('** Selecting flat parameters **')
else:
    shiftN = shiftN_curved
    print(' Curved detector settings ')
    

SET PARAMETERS: 

magnification --> flat: 1.0, curved: 0.893

cell size --> flat: 250, curved: 325

flat --> k: 7, difference from ideal: 0
curved --> k: 9, difference from ideal: 0

artificial feature width --> flat: 20, curved: 26

Chamber diameter --> 18mm = 2923 pixels 

Rolling sobel shift --> flat: 78, curved: 101

 Curved detector settings 


In [633]:
"""


Import image and computed extra variables from settings


"""


Name0_curved_all = focus_im_path.split('/')
if isFlat:
    Name0_curved = Name0_curved_all[-1].replace('.bmp','').replace('.tif','')
else: 
    if isCompressed:
        Name0_curved = Name0_curved_all[-1].replace('.npz','')
    else:
        Name0_curved = Name0_curved_all[-1].replace('.npy','')

## dark
dark = GetDarkFromPath(dark_path, isFlat, isCompressed)

## Curved Detector Parameters
curved_pix_size = curved_pixel/curved_magnification #um
curved_R_chamber = (D/(curved_pix_size*10**(-3)))/2.0 ## radius of the chamber in pixels 


## Flat detector Parameters
flat_pix_size = flat_pixel/flat_magnification #um
flat_size_mm = float(flat_width)*(flat_pix_size*10**(-3)) ## size of the flat detector in mm
flat_height_mm = float(flat_height)*(flat_pix_size*10**(-3)) ## size of the flat detector in mm
flat_size_curved_pix = int(np.round(flat_size_mm/(curved_pix_size*10**(-3)),0)) ## size of the flat detector in curved detector pix
flat_height_curved_pix = int(np.round(flat_height_mm/(curved_pix_size*10**(-3)),0))

flat_R_chamber = (D/(flat_pix_size*10**(-3)))/2.0 ## radius of the chamber in pixels 



## Artifical feature parameters
width_256_curved = width_0_curved
start_0_curved = 25

width_256_flat = width_0_flat
start_0_flat = 25

if isFlat:
    curved_pixel = flat_pixel
    curved_magnification = flat_magnification
    width_0_curved = width_0_flat
    width_256_curved = width_256_flat
    start_0_curved = start_0_flat
    curved_pix_size = flat_pix_size
    curved_width = flat_width
    curved_height = flat_height
    curved_R_chamber = flat_R_chamber

In [634]:
"""

Mask dust


"""

if isFlat:
    im_white = Image.open(white_path)
    im_white = np.array(im_white).astype('float64')
    curved_white_level = np.amax(im_white)
else:
    if isCompressed:
        im_white = np.load(white_path)['arr_0']
        im_white = im_white.astype('float64')
        curved_white_level = np.amax(np.load(saturated_path)['arr_0'])
    else:
        im_white = np.load(white_path).astype('float64')
        curved_white_level = np.amax(np.load(saturated_path))


print('White level curved : ', curved_white_level)

im_white_AF = AddArtificialFeature(im_white, width_0_curved, start_0_curved)
im_white_sobel_AF = Sobel(im_white_AF, curved_sobel_k, "self")
im_white_sobel = RemoveArtificialFeature(im_white_sobel_AF, width_0_curved, start_0_curved, curved_sobel_k)
## Crop sides with artefacts
im_white_sobel = im_white_sobel[:,2*curved_sobel_k:-2*curved_sobel_k]

(Mask_y, Mask_x) = np.where(im_white_sobel>sobel_mask_t)
if MaskDust:
    print('Masking dust')
    mask_white = np.zeros((im_white_sobel.shape))
    for i in range(0, len(Mask_x)):
        x, y = Mask_x[i], Mask_y[i]
        mask_white[y,x] = 1
else:
    if isFlat:
        focus_im = Image.open(focus_im_path)
        focus_im = np.array(focus_im).astype('float64')
    else:
        if isCompressed:
            focus_im = np.load(focus_im_path)['arr_0']
            focus_im = focus_im.astype('float64')
        else:
            focus_im = np.load(focus_im_path).astype('float64')
    focus_im = focus_im[:,2*curved_sobel_k:-2*curved_sobel_k]
    mask_white = np.zeros((focus_im.shape))


if MaskReflections:
    mask_white[y1a:y1b, x1a:x1b] = 1
    mask_white[y2a:y2b, x2a:x2b] = 1
    mask_white[y3a:y3b, x3a:x3b] = 1
    mask_white[y4a:y4b, x4a:x4b] = 1
    

white_sobel_masked = ma.masked_array(im_white_sobel, mask=mask_white)
if maskChamber:
    white_sobel_masked = MaskIm(white_sobel_masked, D_pix)


# ## Plot for testing     
# plt.close()
# plt.figure()
# plt.imshow(mask_white, origin='lower', cmap='gray')
# plt.colorbar()
# plt.show()

White level curved :  2973.0
Masking dust


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [635]:
"""

Load perfect image (sigma extraction)


"""


im_perfect_curved = LoadImPerfect(path_im_perfect_curved, 2**12)
im_perfect_flat = LoadImPerfect(path_im_perfect_flat, 2**12)

# if isFlat:
#     im_perfect_curved = im_perfect_flat
    
plot_range_curved = 300
plot_range_flat = int((plot_range_curved*curved_pixel*curved_magnification)/(flat_pixel*flat_magnification))
(YY, XX) = im_perfect_curved.shape
(YY_flat, XX_flat) = im_perfect_flat.shape

im_perfect_flat_sub = im_perfect_flat[0:plot_range_flat,0:plot_range_flat]
im_perfect_curved_sub = im_perfect_curved[0:plot_range_curved,0:plot_range_curved]

In [637]:
"""


Curved detector


"""

if isFlat:
    print('Careful: Parameters set to flat settings')
else:
    print('Parameters correctly set to curved settings')

Is curved alright


In [639]:
"""
Take Sobel

"""


if isFlat:
    focus_im = Image.open(focus_im_path)
    focus_im = np.array(focus_im).astype('float64')
    
else:
    if isCompressed:
        focus_im = np.load(focus_im_path)['arr_0']
        focus_im = focus_im.astype('float64')
        if DivideDark:
            dark_avg, dark_std = ComputeDark(dark_path)
            focus_im = np.subtract(focus_im, dark_avg)
        if imRescale:
            focus_im = focus_im/np.amax(focus_im)
            focus_im = RescaleImage(focus_im, m_rescale, M_rescale)
        
    else:
        focus_im = np.load(focus_im_path).astype('float64')

## Inverting the Rows
if Inverted:
    focus_im = InvertRows(focus_im)

if maskChamber:
    focus_im = MaskIm(focus_im, D_pix)

focus_im_AF = AddArtificialFeature(focus_im, width_0_curved, start_0_curved)
focus_sobel_AF = Sobel(focus_im_AF, curved_sobel_k, 'self')
focus_sobel = RemoveArtificialFeature(focus_sobel_AF, width_0_curved, start_0_curved, curved_sobel_k)
## Crop sides with artefacts
focus_sobel = focus_sobel[:,2*curved_sobel_k:-2*curved_sobel_k]

## Reload focus im because for some reason AF also added to the original ??
if isFlat:
    focus_im = Image.open(focus_im_path)
    focus_im = np.array(focus_im).astype('float64')
    
else:
    if isCompressed:
        focus_im = np.load(focus_im_path)['arr_0']
        focus_im = focus_im.astype('float64')
        if DivideDark:
            dark_avg, dark_std = ComputeDark(dark_path)
            focus_im = np.subtract(focus_im, dark_avg)
        if imRescale:
            focus_im = focus_im/np.amax(focus_im)
            focus_im = RescaleImage(focus_im, m_rescale, M_rescale)
    else:
        focus_im = np.load(focus_im_path).astype('float64')

## Inverting the Rows
if Inverted:
    focus_im = InvertRows(focus_im)

focus_im = focus_im[:,2*curved_sobel_k:-2*curved_sobel_k]

## Mask the dirt in the image
focus_im = ma.masked_array(focus_im, mask=mask_white)
## Mask the sobel caused by dirt on the detector
focus_sobel = ma.masked_array(focus_sobel, mask=mask_white)

if maskChamber:
    focus_im = MaskIm(focus_im, D_pix)
    focus_sobel = MaskIm(focus_sobel, D_pix)

There are 8 darks
There are 8 darks


In [642]:
"""
Plot image and sobel with cells shown

"""
cell_x, cell_y = curved_cell_x, curved_cell_y
Nx, Ny = SeparateInCells(focus_im, cell_x, cell_y)

# vmax_sobel = 0.45 ## Flat
vmax_sobel = 0.08 ## Curved
# vmax_sobel = 0.03 ## Curved in vivo
# vmax_sobel = 0.003 ## Flat in vivo

## IMAGE
plt.close()
fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(focus_im, origin='lower', cmap='gray') #,vmin=0.0, vmax=1.0
# ax.set_title('Image, cells=('+str(cell_x)+', '+str(cell_y)+')')

(YY, XX) = focus_im.shape

for nx in range(0,Nx-1):
    x_pos = cell_x*(nx+1)
    ax.vlines(x_pos, ymin=0, ymax=(YY-1), colors='blue', lw=0.5, alpha=0.5)

for ny in range(0,Ny-1):
    y_pos = cell_y*(ny+1)
    ax.hlines(y_pos, xmin=0, xmax=(XX-1), colors='blue', lw=0.5, alpha=0.5)
    
if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    ax.add_patch(circ_chamber)
    #     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax.add_patch(square_photonfocus)

# ax.set_xlabel('X [pixel]')
# ax.set_ylabel('Y [pixel]')

# divider = make_axes_locatable(ax)
# cax = divider.append_axes('right', size='5%', pad=0.05)
# fig.colorbar(im, cax=cax, orientation='vertical')

Nticks = 6
stepx = int(np.round(curved_width/(Nticks),0))
Xlist = np.linspace(0,curved_width, curved_width)
Xlist_mm = [i*curved_pixel*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
Xticks_loc.append(curved_width)
Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
Xticks_val.append(np.round(Xlist_mm[-1],1))

stepy = int(np.round(curved_height/(Nticks),0))
Ylist = np.linspace(0,curved_height, curved_height)
Ylist_mm = [i*curved_pixel*10**(-3) for i in Ylist]
Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
Yticks_loc.append(curved_height)
Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(Ylist_mm[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val, fontsize=14)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val, fontsize=14)

ax.set_xlabel('X position [mm]', fontsize=16)
ax.set_ylabel('Y position [mm]', fontsize=16)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

plt.tight_layout()
plt.savefig(saving_path+Name0_curved+'.png', transparent=True)
plt.show()
# plt.close()

## ******************************************************************************************


## SOBEL


plt.close()
fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(focus_sobel, origin='lower', cmap='jet', vmax=vmax_sobel) #,vmin=0.0, vmax=1.0
# ax.set_title('Sobel, k='+str(curved_sobel_k)+', cells=('+str(cell_x)+', '+str(cell_y)+')')


(YY, XX) = focus_sobel.shape

for nx in range(0,Nx-1):
    x_pos = cell_x*(nx+1)
    ax.vlines(x_pos, ymin=0, ymax=(YY-1), lw=0.5, alpha=1.0)

for ny in range(0,Ny-1):
    y_pos = cell_y*(ny+1)
    ax.hlines(y_pos, xmin=0, xmax=(XX-1), lw=0.5, alpha=1.0)
    

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    ax.add_patch(circ_chamber)
#     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax.add_patch(square_photonfocus)


# ax.set_xlabel('X [pixel]')
# ax.set_ylabel('Y [pixel]')
# divider = make_axes_locatable(ax)
# cax = divider.append_axes('right', size='5%', pad=0.05)
# fig.colorbar(im, cax=cax, orientation='vertical')


Nticks = 6
stepx = int(np.round(curved_width/(Nticks),0))
Xlist = np.linspace(0,curved_width, curved_width)
Xlist_mm = [i*curved_pixel*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
Xticks_loc.append(curved_width)
Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
Xticks_val.append(np.round(Xlist_mm[-1],1))

stepy = int(np.round(curved_height/(Nticks),0))
Ylist = np.linspace(0,curved_height, curved_height)
Ylist_mm = [i*curved_pixel*10**(-3) for i in Ylist]
Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
Yticks_loc.append(curved_height)
Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(Ylist_mm[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val, fontsize=14)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val, fontsize=14)

ax.set_xlabel('X position [mm]', fontsize=16)
ax.set_ylabel('Y position [mm]', fontsize=16)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)


plt.tight_layout()
plt.savefig(saving_path+Name0_curved+'_sobel_k'+str(curved_sobel_k)+'_mm.png', transparent=True)
# plt.show()
plt.close()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [643]:
"""
Separate the image in cells

"""

if isFlat==False:
    std_min = 0
    white_min = 0
#     std_min = 150
#     white_min = 2500
else:
    std_min = 0
    white_min = 0

greyscale_sigma_min = 0.1
MinNotMasked = int(0.5*cell_x*cell_y)



cell_x, cell_y = curved_cell_x, curved_cell_y
width, start, kk = width_0_curved, start_0_curved, curved_sobel_k

## *******************************************************************************************************


Nx, Ny = SeparateInCells(focus_im, cell_x, cell_y)

Greyscale_array = np.zeros(focus_im.shape)
Sigma_array_fit = np.zeros(focus_im.shape)
Sigma_array_brute = np.zeros(focus_im.shape)
# sobelDiff_array = np.zeros(focus_im.shape)
sobelMax_array = np.zeros(focus_im.shape)

Gmin_array = np.zeros(focus_im.shape)
Gmax_array = np.zeros(focus_im.shape)

std_array = np.zeros(focus_im.shape)

for nx in tnrange(0,Nx):
    for ny in tnrange(0,Ny):
        cell = GetCell(focus_im, nx, ny, cell_x, cell_y, Nx, Ny)
        cell_whole = cell
        cell_sobel = GetCell(focus_sobel, nx, ny, cell_x, cell_y, Nx, Ny)
        if maskChamber:
            allMasked = cell.mask.all()
            cell = cell.compressed()
            cell_sobel = cell_sobel.compressed()
        else:
            allMasked = False
            
        if allMasked:
            gg = np.nan
            sobel_M = np.nan
            best_sigma = np.nan
            sigma = np.nan
            new_min = np.nan
            new_max = np.nan
            std_val = np.nan
        elif cell_whole.count() < MinNotMasked:
#             print(f'Deleting cell ({nx},{ny})')
            gg = np.nan
            sobel_M = np.nan
            best_sigma = np.nan
            sigma = np.nan
            new_min = np.nan
            new_max = np.nan
            std_val = np.nan
            
        else:
            std_val = np.std(cell)
            ## Get max sobel for this cell (~0.5% of the max to avoid dust)
            sobel_M = GetNMax_perc(cell_sobel, sobel_max)
            ## Get greyscale of cell (% of max, avoid dust)
            flattened_cell = cell.flatten()
            
            if len(flattened_cell)<=((cell_x+1)*(cell_y+1)):
                new_min, new_max = Min_Perc(cell, greyscale_min), Max_Perc(cell, greyscale_max)
            else:
                new_min, new_max = Min_Perc(cell, greyscale_min_largeCell), Max_Perc(cell, greyscale_max)
                

            if isFlat:
                gg = (new_max - new_min)/(2**12-120)
            else:
                gg = (new_max - new_min)/(curved_white_level-120)
            ## if white level too low (too dark), or if the std is too small (no feature), can't compute the sigma
            if new_max<white_min:# or std_val<std_min:
                best_sigma = np.nan
                sigma = np.nan
            else:
                sigma = SigmaFromFit(sobel_M, gg)
                ## Get best sigma fit with this new greyscale by artificallt blurring a theoretical image
                best_sigma, sobel_diff = FindBestSigma(sobel_M, im_perfect_curved_sub, new_min, new_max, width, start, kk)

        for ii in range(0,len(cell_whole[0,:])):
                for jj in range(0,len(cell_whole[:,0])):
                    working_x = cell_x*nx + ii
                    working_y = cell_y*ny + jj
                    Greyscale_array[working_y, working_x] = gg
                    sobelMax_array[working_y, working_x] = sobel_M
                    Sigma_array_brute[working_y, working_x] = best_sigma
                    Sigma_array_fit[working_y, working_x] = sigma
                    Gmin_array[working_y, working_x] = new_min
                    Gmax_array[working_y, working_x] = new_max
                    std_array[working_y, working_x] = std_val


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [644]:
# """

# Standard deviation

# """


# fig = plt.figure(figsize=(9,7))
# ax = fig.add_subplot(1, 1, 1)

# im = ax.imshow(std_array, origin='lower', cmap='viridis')

# if isFlat==False:
#     circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
#                                   edgecolor='indianred', facecolor='None')
#     ax.add_patch(circ_chamber)
#     #     ## vertical flat sensor
# #     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
# #                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
# #                                            edgecolor='limegreen', facecolor='None')
#     ## horizontal flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
#                                            flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
#     ax.add_patch(square_photonfocus)


# Nticks = 6
# stepx = int(np.round(curved_width/(Nticks),0))
# Xlist = np.linspace(0,curved_width, curved_width)
# Xlist_mm = [i*curved_pixel*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
# Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
# Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
# Xticks_loc.append(curved_width)
# Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
# Xticks_val.append(np.round(Xlist_mm[-1],1))

# stepy = int(np.round(curved_height/(Nticks),0))
# Ylist = np.linspace(0,curved_height, curved_height)
# Ylist_mm = [i*curved_pixel*10**(-3) for i in Ylist]
# Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
# Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
# Yticks_loc.append(curved_height)
# Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
# Yticks_val.append(np.round(Ylist_mm[-1],1))

# ax.set_xticks(Xticks_loc)
# ax.set_xticklabels(Xticks_val, fontsize=14)
# ax.set_yticks(Yticks_loc)
# ax.set_yticklabels(Yticks_val, fontsize=14)

# ax.set_xlabel('X position [mm]', fontsize=16)
# ax.set_ylabel('Y position [mm]', fontsize=16)

# divider = make_axes_locatable(ax)
# cax = divider.append_axes('right', size='5%', pad=0.05)
# cbar = fig.colorbar(im, cax=cax, orientation='vertical')
# cbar.ax.tick_params(labelsize=14)

# plt.tight_layout()
# if Inverted:
#     plt.savefig(saving_path+Name0_curved+'_std.png', transparent=True)
# else:
#     plt.savefig(saving_path+Name0_curved+'_std.png', transparent=True)

# plt.show()
# # plt.close()

In [573]:
"""

Looking at min and max values of greyscale (debug mode)


"""


plt.close()

fig, ax = plt.subplots(2, 1, figsize=(7,10))
im0 = ax[0].imshow(Gmin_array, origin='lower', cmap='gray')

divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im0, cax=cax, orientation='vertical')

im1 = ax[1].imshow(Gmax_array, origin='lower', cmap='gray')

divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im1, cax=cax, orientation='vertical')

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
                                           flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='black', facecolor='None')
    ax[0].add_patch(circ_chamber)
    ax[0].add_patch(square_photonfocus)
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    #     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax[1].add_patch(circ_chamber)
    ax[1].add_patch(square_photonfocus)



ax[0].set_title('Min value')
ax[1].set_title('Max value')
plt.tight_layout()
plt.savefig(saving_path+Name0_curved+'_Gmin'+str(greyscale_min)+'_Gmax'+str(greyscale_max)+'.png', transparent=True)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [575]:
"""


Plot sigma (brute force method - With physical units)


"""


print((curved_pixel/curved_magnification), 'um per pixel')


## Standardized range imposed for all plots
min_brute_not_r = 10
max_brute_not_r = 70


plt.close()
fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(Sigma_array_brute*(curved_pixel/curved_magnification), origin='lower', 
               cmap='magma_r', vmin=min_brute_not_r, vmax=max_brute_not_r) #, vmin=20  'inferno_r' , vmin=20, vmax=60

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='limegreen', facecolor='None')
    ax.add_patch(circ_chamber)
    #     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax.add_patch(square_photonfocus)

# circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, 
#                               edgecolor='limegreen', facecolor='None')
# ax.add_patch(circ_chamber)
# square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                        flat_size_curved_pix, flat_size_curved_pix, alpha=0.7, 
#                                        edgecolor='black', facecolor='None')
# ax.add_patch(square_photonfocus)

# ax.set_title(' Sigma (blurring) in micrometers')
# ax.set_xlabel('X [pixel]')
# ax.set_ylabel('Y [pixel]')
# divider = make_axes_locatable(ax)
# cax = divider.append_axes('right', size='5%', pad=0.05)
# fig.colorbar(im, cax=cax, orientation='vertical')
Nticks = 6
stepx = int(np.round(curved_width/(Nticks),0))
Xlist = np.linspace(0,curved_width, curved_width)
Xlist_mm = [i*curved_pixel*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
Xticks_loc.append(curved_width)
Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
Xticks_val.append(np.round(Xlist_mm[-1],1))

stepy = int(np.round(curved_height/(Nticks),0))
Ylist = np.linspace(0,curved_height, curved_height)
Ylist_mm = [i*curved_pixel*10**(-3) for i in Ylist]
Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
Yticks_loc.append(curved_height)
Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(Ylist_mm[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val, fontsize=14)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val, fontsize=14)

ax.set_xlabel('X position [mm]', fontsize=16)
ax.set_ylabel('Y position [mm]', fontsize=16)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

plt.tight_layout()
plt.savefig(saving_path+Name0_curved+'_UM_sigmasBrute_k'+str(curved_sobel_k)+'_mm_StandardScale.png', transparent=True)

plt.show()


6.159014557670773 um per pixel
Brute sigma range between 10 - 70


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [645]:
"""


Rolling sobel


"""

cell_x, cell_y = curved_cell_x, curved_cell_y
width, start, kk = width_0_curved, start_0_curved, curved_sobel_k

MinNotMasked = int(0.5*cell_x*cell_y)

### *******************************************************************************************************


Nx, Ny = SeparateInCells(focus_im, cell_x, cell_y)

rGreyscale_array = np.zeros(focus_im.shape)
rGreyscale_array[:] = np.nan
rSigma_array_brute = np.zeros(focus_im.shape)
rSigma_array_brute[:] = np.nan

rsobelMax_array = np.zeros(focus_im.shape)
rsobelMax_array[:] = np.nan

rstd = np.zeros(focus_im.shape)
rstd[:] = np.nan

rGmin_array = np.zeros(focus_im.shape)
rGmin_array[:] = np.nan
rGmax_array = np.zeros(focus_im.shape)
rGmax_array[:] = np.nan


try:
    for i in tnrange(0, (XX-2*shiftN), shiftN, desc='i'):
        for j in tnrange(0, (YY-2*shiftN), shiftN, desc='j'):
#             print(f'i={i}, j={j}')
            x_start, x_end = i, i+cell_x
            y_start, y_end = j, j+cell_y
            cell = focus_im[y_start:y_end, x_start:x_end]
            cell_whole = cell
            cell_sobel = focus_sobel[y_start:y_end, x_start:x_end]
            if maskChamber:
                allMasked = cell.mask.all()
                cell = cell.compressed()
                cell_sobel = cell_sobel.compressed()
            else:
                allMasked = False
            
            if allMasked:
                gg = np.nan
                sobel_M = np.nan
                best_sigma = np.nan
                sigma = np.nan
                new_min = np.nan
                new_max = np.nan
                std_val = np.nan
            elif cell_whole.count() < MinNotMasked:
#                 print(f'Deleting cell ({nx},{ny})')
                gg = np.nan
                sobel_M = np.nan
                best_sigma = np.nan
                sigma = np.nan
                new_min = np.nan
                new_max = np.nan
                std_val = np.nan
            
            else:
                ## Get standard deviation of pixels in each cells
                std_val = np.std(cell)
            
                ## Get max sobel for this cell (~0.5% of the max to avoid dust)
                sobel_M = GetNMax_perc(cell_sobel, sobel_max)
                ## Get greyscale of cell (% of max, avoid dust)
                flattened_cell = cell.flatten()
                new_min_prev, new_max_prev = new_min, new_max
                if len(flattened_cell)<=((cell_x+1)*(cell_y+1)):
                    new_min, new_max = Min_Perc(cell, greyscale_min), Max_Perc(cell, greyscale_max)
                else:
                    new_min, new_max = Min_Perc(cell, greyscale_min_largeCell), Max_Perc(cell, greyscale_max)
        
                if isFlat:
                    if new_min>2000:
                        new_min = new_min_prev
                    gg = (new_max - new_min)/(2**12-120)
                else:
                    gg = (new_max - new_min)/(curved_white_level-120)
                    
                if gg<greyscale_sigma_min:
                    ## Brute force best sigma (more reliable, slower)
                    best_sigma, sobel_diff = FindBestSigma(sobel_M, im_perfect_curved_sub, new_min, new_max, width, start, kk)
                else:
                    ## Brute force best sigma (more reliable, slower)
                    best_sigma, sobel_diff = FindBestSigma(sobel_M, im_perfect_curved_sub, new_min, new_max, width, start, kk)

            ## Now populate arrays
            xmid, ymid = int(i+cell_x/2), int(j+cell_y/2)
            rxstart, rxend = int(xmid-shiftN/2), int(xmid+shiftN/2)
            rystart, ryend = int(ymid-shiftN/2), int(ymid+shiftN/2)
        
            rGreyscale_array[rystart:ryend,rxstart:rxend] = gg
            rsobelMax_array[rystart:ryend,rxstart:rxend] = sobel_M
            rSigma_array_brute[rystart:ryend,rxstart:rxend] = best_sigma
            rGmin_array[rystart:ryend,rxstart:rxend] = new_min
            rGmax_array[rystart:ryend,rxstart:rxend] = new_max
            rstd[rystart:ryend,rxstart:rxend] = std_val
except KeyboardInterrupt:  
#     pass
    pbar.close()

i:   0%|          | 0/39 [00:00<?, ?it/s]

j:   0%|          | 0/29 [00:00<?, ?it/s]

j:   0%|          | 0/29 [00:00<?, ?it/s]

j:   0%|          | 0/29 [00:00<?, ?it/s]

j:   0%|          | 0/29 [00:00<?, ?it/s]

...

j:   0%|          | 0/29 [00:00<?, ?it/s]

j:   0%|          | 0/29 [00:00<?, ?it/s]

j:   0%|          | 0/29 [00:00<?, ?it/s]

j:   0%|          | 0/29 [00:00<?, ?it/s]

j:   0%|          | 0/29 [00:00<?, ?it/s]

In [646]:
"""

Save results from rolling cell method

"""

np.save(saving_path+Name0_curved+'_r_sigmabrute.npy', rSigma_array_brute)
np.save(saving_path+Name0_curved+'_r_greyscale.npy', rGreyscale_array)
np.save(saving_path+Name0_curved+'_r_sobelMax.npy', rsobelMax_array)
np.save(saving_path+Name0_curved+'_r_Gmin.npy', rGmin_array)
np.save(saving_path+Name0_curved+'_r_Gmax.npy', rGmax_array)

In [647]:
"""

Separate in cells


"""


# if isFlat:
#     sobel_max = 0.005
# else:
#     sobel_max = 0.02
# greyscale_min, greyscale_max = 0.0005, 0.001 #0.00014
# greyscale_min_largeCell = 0.00007

## *******************************************************************************************************


Nx, Ny = SeparateInCells(focus_im, cell_x, cell_y)

Greyscale_array = np.zeros(focus_im.shape)
Sigma_array_fit = np.zeros(focus_im.shape)
Sigma_array_brute = np.zeros(focus_im.shape)
# sobelDiff_array = np.zeros(focus_im.shape)
sobelMax_array = np.zeros(focus_im.shape)

Gmin_array = np.zeros(focus_im.shape)
Gmax_array = np.zeros(focus_im.shape)

std_array = np.zeros(focus_im.shape)

for nx in tnrange(0,Nx):
    for ny in tnrange(0,Ny):
        cell = GetCell(focus_im, nx, ny, cell_x, cell_y, Nx, Ny)
        cell_whole = cell
        cell_sobel = GetCell(focus_sobel, nx, ny, cell_x, cell_y, Nx, Ny)
        if maskChamber:
            allMasked = cell.mask.all()
            cell = cell.compressed()
            cell_sobel = cell_sobel.compressed()
        else:
            allMasked = False
            
        if allMasked:
            gg = np.nan
            sobel_M = np.nan
            best_sigma = np.nan
            sigma = np.nan
            new_min = np.nan
            new_max = np.nan
            std_val = np.nan
        elif cell_whole.count() < MinNotMasked:
#             print(f'Deleting cell ({nx},{ny})')
            gg = np.nan
            sobel_M = np.nan
            best_sigma = np.nan
            sigma = np.nan
            new_min = np.nan
            new_max = np.nan
            std_val = np.nan
            
        else:
            std_val = np.std(cell)
            ## Get max sobel for this cell (~0.5% of the max to avoid dust)
            sobel_M = GetNMax_perc(cell_sobel, sobel_max)
            ## Get greyscale of cell (% of max, avoid dust)
            flattened_cell = cell.flatten()
            
            if len(flattened_cell)<=((cell_x+1)*(cell_y+1)):
                new_min, new_max = Min_Perc(cell, greyscale_min), Max_Perc(cell, greyscale_max)
            else:
                new_min, new_max = Min_Perc(cell, greyscale_min_largeCell), Max_Perc(cell, greyscale_max)
                

            if isFlat:
                gg = (new_max - new_min)/(2**12-120)
            else:
                gg = (new_max - new_min)/(curved_white_level-120)
#             if gg<greyscale_sigma_min:
#                 if new_max<white_min or std_val<std_min:
#                     best_sigma = np.nan
#                     sigma = np.nan
#                 else:
#                     sigma = SigmaFromFit(sobel_M, gg)
#                     ## Get best sigma fit with this new greyscale by artificallt blurring a theoretical image
#                     best_sigma, sobel_diff = FindBestSigma(sobel_M, im_perfect_curved_sub, new_min, new_max, width, start, kk)
#             else:


            ## if white level too low (too dark), or if the std is too small (no feature), can't compute the sigma
            if new_max<white_min:# or std_val<std_min:
                best_sigma = np.nan
                sigma = np.nan
            else:
                sigma = SigmaFromFit(sobel_M, gg)
                ## Get best sigma fit with this new greyscale by artificallt blurring a theoretical image
                best_sigma, sobel_diff = FindBestSigma(sobel_M, im_perfect_curved_sub, new_min, new_max, width, start, kk)

    #         Greyscale = new_max - new_min
    #         Greyscale = MichelsonContrast(focus_im)
    #         Greyscale = Range_Perc(cell, 0.01)
    #         Greyscale = Max_Perc(cell, 0.01)
    #         sigma = GetSigma(sobel_M, Michelson, a, b, c, m, n, d)

        for ii in range(0,len(cell_whole[0,:])):
                for jj in range(0,len(cell_whole[:,0])):
                    working_x = cell_x*nx + ii
                    working_y = cell_y*ny + jj
                    Greyscale_array[working_y, working_x] = gg
                    sobelMax_array[working_y, working_x] = sobel_M
                    Sigma_array_brute[working_y, working_x] = best_sigma
                    Sigma_array_fit[working_y, working_x] = sigma
                    Gmin_array[working_y, working_x] = new_min
                    Gmax_array[working_y, working_x] = new_max
                    std_array[working_y, working_x] = std_val

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [648]:
"""



Plot a single figure with everything all at once




"""
ff_title = 32
ff_labels = 26
ff_letters = 40

cell_x, cell_y = curved_cell_x, curved_cell_y
Nx, Ny = SeparateInCells(focus_im, cell_x, cell_y)

# vmax_sobel = 0.45 ## Flat
vmax_sobel = 0.08 ## Curved
# vmax_sobel = 0.13 ## Curved
# vmax_sobel = 0.03 ## Curved in vivo
# vmax_sobel = 0.003 ## Flat in vivo

# Nticks = 6
# stepx = int(np.round(curved_width/(Nticks),0))
# Xlist = np.linspace(0,curved_width, curved_width)
# Xlist_mm = [i*curved_pixel*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
# Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
# Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
# Xticks_loc.append(curved_width)
# Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
# Xticks_val.append(np.round(Xlist_mm[-1],1))

# stepy = int(np.round(curved_height/(Nticks),0))
# Ylist = np.linspace(0,curved_height, curved_height)
# Ylist_mm = [i*curved_pixel*10**(-3) for i in Ylist]
# Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
# Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
# Yticks_loc.append(curved_height)
# Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
# Yticks_val.append(np.round(Ylist_mm[-1],1))

Nticks = 6
stepx = int(np.round(curved_width/(Nticks),0))
Xlist = np.linspace(0,curved_width, curved_width)
Xlist_mm = [i*curved_pixel/curved_magnification*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
Xticks_loc.append(curved_width)
Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
Xticks_val.append(np.round(Xlist_mm[-1],1))

stepy = int(np.round(curved_height/(Nticks),0))
Ylist = np.linspace(0,curved_height, curved_height)
Ylist_mm = [i*curved_pixel/curved_magnification*10**(-3) for i in Ylist]
Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
Yticks_loc.append(curved_height)
Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(Ylist_mm[-1],1))



plt.close()

## Large
fig, ax = plt.subplots(2,3, figsize=(25, 12))
## Smaller (paper)
# fig, ax = plt.subplots(2,3, figsize=(15, 9))

## IMAGE
im00 = ax[0,0].imshow(focus_im, origin='lower', cmap='gray') #,vmin=0.0, vmax=1.0
ax[0,0].set_title('A', fontsize=ff_letters, loc='left', fontweight='bold')

(YY, XX) = focus_im.shape
for nx in range(0,Nx-1):
    x_pos = cell_x*(nx+1)
    ax[0,0].vlines(x_pos, ymin=0, ymax=(YY-1), colors='blue', lw=0.5, alpha=0.5)

for ny in range(0,Ny-1):
    y_pos = cell_y*(ny+1)
    ax[0,0].hlines(y_pos, xmin=0, xmax=(XX-1), colors='blue', lw=0.5, alpha=0.5)
    
if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    ax[0,0].add_patch(circ_chamber)
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax[0,0].add_patch(square_photonfocus)

ax[0,0].set_xticks(Xticks_loc)
ax[0,0].set_xticklabels(Xticks_val, fontsize=ff_labels)
ax[0,0].set_yticks(Yticks_loc)
ax[0,0].set_yticklabels(Yticks_val, fontsize=ff_labels)

# ax[0,0].set_xlabel('X position [mm]', fontsize=ff_title)
ax[0,0].set_ylabel('Y position [mm]', fontsize=ff_title)

divider = make_axes_locatable(ax[0,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im00, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=ff_labels)



## SOBEL

im01 = ax[0,1].imshow(focus_sobel, origin='lower', cmap='jet', vmax=vmax_sobel) #,vmin=0.0, vmax=1.0
ax[0,1].set_title('B', fontsize=ff_letters, loc='left', fontweight='bold')

(YY, XX) = focus_sobel.shape
for nx in range(0,Nx-1):
    x_pos = cell_x*(nx+1)
    ax[0,1].vlines(x_pos, ymin=0, ymax=(YY-1), lw=0.5, alpha=1.0)

for ny in range(0,Ny-1):
    y_pos = cell_y*(ny+1)
    ax[0,1].hlines(y_pos, xmin=0, xmax=(XX-1), lw=0.5, alpha=1.0)
    

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    ax[0,1].add_patch(circ_chamber)
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax[0,1].add_patch(square_photonfocus)


ax[0,1].set_xticks(Xticks_loc)
ax[0,1].set_xticklabels(Xticks_val, fontsize=ff_labels)
ax[0,1].set_yticks(Yticks_loc)
ax[0,1].set_yticklabels(Yticks_val, fontsize=ff_labels)

# ax[0,1].set_xlabel('X position [mm]', fontsize=ff_title)
# ax[0,1].set_ylabel('Y position [mm]', fontsize=ff_title)

divider = make_axes_locatable(ax[0,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im01, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=ff_labels)



# MAX SOBEL
im02 = ax[0,2].imshow(sobelMax_array, origin='lower', cmap='jet')
ax[0,2].set_title('C', fontsize=ff_letters, loc='left', fontweight='bold')

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    ax[0,2].add_patch(circ_chamber)
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax[0,2].add_patch(square_photonfocus)

ax[0,2].set_xticks(Xticks_loc)
ax[0,2].set_xticklabels(Xticks_val, fontsize=ff_labels)
ax[0,2].set_yticks(Yticks_loc)
ax[0,2].set_yticklabels(Yticks_val, fontsize=ff_labels)

# ax[0,2].set_xlabel('X position [mm]', fontsize=ff_title)
# ax[0,2].set_ylabel('Y position [mm]', fontsize=ff_title)

divider = make_axes_locatable(ax[0,2])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im02, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=ff_labels)


## Greyscale
im10 = ax[1,0].imshow(Greyscale_array, origin='lower', cmap='gray'  ) #,vmin=0.0, vmax=1.0, vmin=0.0, vmax=0.35
ax[1,0].set_title('D', fontsize=ff_letters, loc='left', fontweight='bold')

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    ax[1,0].add_patch(circ_chamber)
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax[1,0].add_patch(square_photonfocus)


ax[1,0].set_xticks(Xticks_loc)
ax[1,0].set_xticklabels(Xticks_val, fontsize=ff_labels)
ax[1,0].set_yticks(Yticks_loc)
ax[1,0].set_yticklabels(Yticks_val, fontsize=ff_labels)

ax[1,0].set_xlabel('X position [mm]', fontsize=ff_title)
ax[1,0].set_ylabel('Y position [mm]', fontsize=ff_title)

divider = make_axes_locatable(ax[1,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im10, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=ff_labels)



## Sigma

vmin_sigma = 35
vmax_sigma = 90


im11 = ax[1,1].imshow(Sigma_array_brute*(curved_pixel/curved_magnification), origin='lower', cmap='magma_r', vmin=vmin_sigma, vmax=vmax_sigma) #,vmin=1.0,vmax=10
ax[1,1].set_title('E', fontsize=ff_letters, loc='left', fontweight='bold')

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='limegreen', facecolor='None')
    ax[1,1].add_patch(circ_chamber)
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax[1,1].add_patch(square_photonfocus)

ax[1,1].set_xticks(Xticks_loc)
ax[1,1].set_xticklabels(Xticks_val, fontsize=ff_labels)
ax[1,1].set_yticks(Yticks_loc)
ax[1,1].set_yticklabels(Yticks_val, fontsize=ff_labels)

ax[1,1].set_xlabel('X position [mm]', fontsize=ff_title)
# ax[1,1].set_ylabel('Y position [mm]', fontsize=ff_title)

divider = make_axes_locatable(ax[1,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im11, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=ff_labels)



im12 = ax[1,2].imshow(rSigma_array_brute*(curved_pixel/curved_magnification), origin='lower', cmap='magma_r', vmin=vmin_sigma, vmax=vmax_sigma)# , vmin=min_brute_not_r, vmax=max_brute_not_r
ax[1,2].set_title('F', fontsize=ff_letters, loc='left', fontweight='bold')

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='limegreen', facecolor='None')
    ax[1,2].add_patch(circ_chamber)
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax[1,2].add_patch(square_photonfocus)

ax[1,2].set_xticks(Xticks_loc)
ax[1,2].set_xticklabels(Xticks_val, fontsize=ff_labels)
ax[1,2].set_yticks(Yticks_loc)
ax[1,2].set_yticklabels(Yticks_val, fontsize=ff_labels)

ax[1,2].set_xlabel('X position [mm]', fontsize=ff_title)
# ax[1,2].set_ylabel('Y position [mm]', fontsize=ff_title)

divider = make_axes_locatable(ax[1,2])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im12, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=ff_labels)

plt.tight_layout()
# plt.savefig(saving_path+Name0_curved+'_AllFigures_LessLabels.png', transparent=True)
plt.show()
# plt.close()

## ******************************************************************************************


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [649]:
"""



Min and Max greyscale - rolling sobel


"""

print(f'greyscale_min={greyscale_min}')
plt.close()

fig, ax = plt.subplots(2, 1, figsize=(7,10))

im0 = ax[0].imshow(rGmin_array, origin='lower', cmap='gray')

divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im0, cax=cax, orientation='vertical')

im1 = ax[1].imshow(rGmax_array, origin='lower', cmap='gray')

divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im1, cax=cax, orientation='vertical')

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax[0].add_patch(circ_chamber)
    ax[0].add_patch(square_photonfocus)
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    #     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax[1].add_patch(circ_chamber)
    ax[1].add_patch(square_photonfocus)



ax[0].set_title('Min value')
ax[1].set_title('Max value')
plt.tight_layout()
plt.savefig(saving_path+Name0_curved+'_Rolling_Gmin'+str(greyscale_min)+'_Gmax'+str(greyscale_max)+'.png', transparent=True)

plt.show()


greyscale_min=0.0005


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [650]:
"""

Standard deviation/cell

"""

fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(rstd, origin='lower', cmap='viridis')

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    ax.add_patch(circ_chamber)
    #     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax.add_patch(square_photonfocus)

Nticks = 6
stepx = int(np.round(curved_width/(Nticks),0))
Xlist = np.linspace(0,curved_width, curved_width)
Xlist_mm = [i*curved_pixel*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
Xticks_loc.append(curved_width)
Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
Xticks_val.append(np.round(Xlist_mm[-1],1))

stepy = int(np.round(curved_height/(Nticks),0))
Ylist = np.linspace(0,curved_height, curved_height)
Ylist_mm = [i*curved_pixel*10**(-3) for i in Ylist]
Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
Yticks_loc.append(curved_height)
Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(Ylist_mm[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val, fontsize=14)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val, fontsize=14)

ax.set_xlabel('X position [mm]', fontsize=16)
ax.set_ylabel('Y position [mm]', fontsize=16)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

plt.tight_layout()
# plt.savefig(saving_path+Name0_curved+'_r'+str(shiftN)+'_std.png', transparent=True)

plt.show()
# plt.close()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [651]:
'''


Min and max values of greyscale


'''

plt.close()

fig, ax = plt.subplots(2, 1, figsize=(7,10))

im0 = ax[0].imshow(rGmin_array, origin='lower', cmap='gray')

divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im0, cax=cax, orientation='vertical')

im1 = ax[1].imshow(rGmax_array, origin='lower', cmap='gray')

divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im1, cax=cax, orientation='vertical')

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
                                           flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='black', facecolor='None')
    ax[0].add_patch(circ_chamber)
    ax[0].add_patch(square_photonfocus)
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    #     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax[1].add_patch(circ_chamber)
    ax[1].add_patch(square_photonfocus)



ax[0].set_title('Min value')
ax[1].set_title('Max value')
plt.tight_layout()
# plt.savefig(saving_path+Name0_curved+'_Gmin_k'+str(greyscale_min)+'_'+str(greyscale_max)+'.png', transparent=True)

plt.show()


print(greyscale_min)
print(greyscale_min*cell_x*cell_y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0005
52.8125


In [652]:
## Plot results Rolling sobel

## *********************************************************************************************
##
##    Max Sobel

fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(rsobelMax_array, origin='lower', cmap='jet')

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    ax.add_patch(circ_chamber)
    #     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax.add_patch(square_photonfocus)

Nticks = 6
stepx = int(np.round(curved_width/(Nticks),0))
Xlist = np.linspace(0,curved_width, curved_width)
Xlist_mm = [i*curved_pixel*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
Xticks_loc.append(curved_width)
Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
Xticks_val.append(np.round(Xlist_mm[-1],1))

stepy = int(np.round(curved_height/(Nticks),0))
Ylist = np.linspace(0,curved_height, curved_height)
Ylist_mm = [i*curved_pixel*10**(-3) for i in Ylist]
Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
Yticks_loc.append(curved_height)
Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(Ylist_mm[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val, fontsize=14)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val, fontsize=14)

ax.set_xlabel('X position [mm]', fontsize=16)
ax.set_ylabel('Y position [mm]', fontsize=16)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

plt.tight_layout()
plt.savefig(saving_path+Name0_curved+'_r'+str(shiftN)+'_maxSobel_k'+str(curved_sobel_k)+'_mm.png', transparent=True)

# plt.show()
plt.close()


## *********************************************************************************************
##
##    Greyscale


fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(rGreyscale_array, origin='lower', cmap='gray') #,vmin=0.0, vmax=1.0, vmin=0.0, vmax=0.35


if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='indianred', facecolor='None')
    ax.add_patch(circ_chamber)
    #     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax.add_patch(square_photonfocus)


Nticks = 6
stepx = int(np.round(curved_width/(Nticks),0))
Xlist = np.linspace(0,curved_width, curved_width)
Xlist_mm = [i*curved_pixel*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
Xticks_loc.append(curved_width)
Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
Xticks_val.append(np.round(Xlist_mm[-1],1))

stepy = int(np.round(curved_height/(Nticks),0))
Ylist = np.linspace(0,curved_height, curved_height)
Ylist_mm = [i*curved_pixel*10**(-3) for i in Ylist]
Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
Yticks_loc.append(curved_height)
Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(Ylist_mm[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val, fontsize=14)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val, fontsize=14)

ax.set_xlabel('X position [mm]', fontsize=16)
ax.set_ylabel('Y position [mm]', fontsize=16)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

plt.tight_layout()
plt.savefig(saving_path+Name0_curved+'_r'+str(shiftN)+'_greyscale_k'+str(curved_sobel_k)+'_mm.png', transparent=True)

# plt.show()
plt.close()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [653]:
## *********************************************************************************************
##
##    Sigma - brute
##       With physical units
##  rolling cells
##
 
min_brute_not_r = 10
max_brute_not_r = 70

print(f'Brute sigma range between {min_brute_not_r} - {max_brute_not_r}')


plt.close()
fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(rSigma_array_brute*(curved_pixel/curved_magnification), origin='lower', 
               cmap='magma_r', vmin=min_brute_not_r, vmax=max_brute_not_r) #, vmin=20  'inferno_r' , vmin=20, vmax=60

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='limegreen', facecolor='None')
    ax.add_patch(circ_chamber)
    #     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax.add_patch(square_photonfocus)

Nticks = 6
stepx = int(np.round(curved_width/(Nticks),0))
Xlist = np.linspace(0,curved_width, curved_width)
Xlist_mm = [i*curved_pixel*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
Xticks_loc.append(curved_width)
Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
Xticks_val.append(np.round(Xlist_mm[-1],1))

stepy = int(np.round(curved_height/(Nticks),0))
Ylist = np.linspace(0,curved_height, curved_height)
Ylist_mm = [i*curved_pixel*10**(-3) for i in Ylist]
Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
Yticks_loc.append(curved_height)
Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(Ylist_mm[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val, fontsize=14)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val, fontsize=14)

ax.set_xlabel('X position [mm]', fontsize=16)
ax.set_ylabel('Y position [mm]', fontsize=16)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

plt.tight_layout()
plt.savefig(saving_path+Name0_curved+'_r'+str(shiftN)+'_UM_sigmasBrute_k'+str(curved_sobel_k)+'_mm_StandardScale.png', transparent=True)

# plt.show()
plt.close()


6.159014557670773 um per pixel
Brute sigma range between 10 - 70


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
## *********************************************************************************************
##
##    Sigma - fit
##       With physical units
##

    
    
min_fit_not_r = 10
max_fit_not_r = 70
print(f'Fit sigma range between {min_fit_not_r} - {max_fit_not_r}')

fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(rSigma_array_fit*(curved_pixel/curved_magnification), origin='lower', 
               cmap='magma_r', vmin=min_fit_not_r, vmax=max_fit_not_r) #, vmin=20  'inferno_r' , vmin=20, vmax=60

if isFlat==False:
    circ_chamber = patches.Circle((XX/2, YY/2), curved_R_chamber, alpha=0.7, lw=2.0,
                                  edgecolor='limegreen', facecolor='None')
    ax.add_patch(circ_chamber)
    #     ## vertical flat sensor
#     square_photonfocus = patches.Rectangle((XX/2-flat_height_curved_pix/2, YY/2-flat_size_curved_pix/2),
#                                            flat_height_curved_pix, flat_size_curved_pix, alpha=0.7, lw=2.0, 
#                                            edgecolor='limegreen', facecolor='None')
    ## horizontal flat sensor
    square_photonfocus = patches.Rectangle((XX/2-flat_size_curved_pix/2, YY/2-flat_height_curved_pix/2),
                                           flat_size_curved_pix, flat_height_curved_pix, alpha=0.7, lw=2.0, 
                                           edgecolor='limegreen', facecolor='None')
    ax.add_patch(square_photonfocus)

Nticks = 6
stepx = int(np.round(curved_width/(Nticks),0))
Xlist = np.linspace(0,curved_width, curved_width)
Xlist_mm = [i*curved_pixel*10**(-3) for i in Xlist] #- (curved_width*curved_pixel*10**(-3))/2
Xlist_mm = Xlist_mm-max(Xlist_mm)/2 #max(Xlist_mm)/2+0.01
Xticks_loc = [i*1.0/Nticks*curved_width for i in range(0,Nticks)]
Xticks_loc.append(curved_width)
Xticks_val = [np.round(Xlist_mm[i*stepx],1) for i in range(0,Nticks)]
Xticks_val.append(np.round(Xlist_mm[-1],1))

stepy = int(np.round(curved_height/(Nticks),0))
Ylist = np.linspace(0,curved_height, curved_height)
Ylist_mm = [i*curved_pixel*10**(-3) for i in Ylist]
Ylist_mm = Ylist_mm-(max(Ylist_mm)/2)
Yticks_loc = [i*1.0/Nticks*curved_height for i in range(0,Nticks)]
Yticks_loc.append(curved_height)
Yticks_val = [np.round(Ylist_mm[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(Ylist_mm[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val, fontsize=14)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val, fontsize=14)

ax.set_xlabel('X position [mm]', fontsize=16)
ax.set_ylabel('Y position [mm]', fontsize=16)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar = fig.colorbar(im, cax=cax, orientation='vertical')
cbar.ax.tick_params(labelsize=14)

plt.tight_layout()
plt.savefig(saving_path+Name0_curved+'_r'+str(shiftN)+'_UM_sigmasFit_k'+str(curved_sobel_k)+'_StandardScale.png', transparent=True)

# plt.show()
plt.close()
